In [4]:
import numpy as np
import pandas as pd

In [5]:
#import iris data
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
  
# metadata 
print(iris.metadata) 
  
# variable information 
print(iris.variables) 

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'published_in': 'Significance, 2021', 'year': 2021, 'url': 'https://www.semanticscholar.org/paper/4599862ea877863669a6a8e63a3c707a787d5d7e', 'doi': '1740-9713.01589'}, 'add

In [11]:
#split dataset into shadow and target datasets

from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# "shadow" dataset
shadow_X, shadow_y = X_train, y_train

# "target" dataset
target_X, target_y = X_test, y_test

In [12]:
#create class for model creation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class CustomLogisticRegressionModel:
    def __init__(self, X=None, y=None):
        self.model = LogisticRegression(max_iter=1000)
        self.X = X
        self.y = y
        
    def train(self, X_train=None, y_train=None):
        if X_train is not None and y_train is not None:
            self.X = X_train
            self.y = y_train
        self.model.fit(self.X, self.y)
        
    def evaluate(self, X_test, y_test):
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        return accuracy

In [15]:
#create original target model
X_train, X_test, y_train, y_test = train_test_split(target_X, target_y, test_size=0.2, random_state=42)

target_original_model = CustomLogisticRegressionModel(X_train, y_train)
target_original_model.train()

# Evaluate the model with testing data
test_accuracy = target_original_model.evaluate(X_test, y_test)

print(f"Testing accuracy: {test_accuracy}")

Testing accuracy: 1.0


/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
#create unlearned target model
index_unlearned = np.random.randint(0, len(target_X))

target_unlearned_X = target_X.drop(index=index_unlearned)
target_unlearned_y = target_y.drop(index=index_unlearned)

X_train_unlearned, X_test_unlearned, y_train_unlearned, y_test_unlearned = train_test_split(target_unlearned_X, target_unlearned_y, test_size=0.2, random_state=42)

target_unlearned_model = CustomLogisticRegressionModel(X_train_unlearned, y_train_unlearned)
target_unlearned_model.train()

# Evaluate the model with testing data
test_accuracy_unlearned = target_unlearned_model.evaluate(X_test_unlearned, y_test_unlearned)

print(f"Testing accuracy: {test_accuracy_unlearned}")

Testing accuracy: 1.0


/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
#split shadow dataset into positive and negative

#here,the code does a 80-20 split (positive recieves 80% and negative recieves 20%)
shadow_X_train, shadow_X_test, shadow_y_train, shadow_y_test = train_test_split(shadow_X, shadow_y, test_size=0.2, random_state=42)

#shadow "positive"
positive_shadow_X, positive_shadow_y = shadow_X_train, shadow_y_train

#shadow "negative"
negative_target_X, negative_target_y = shadow_X_test, shadow_y_test

In [22]:
#create original shadow model

X_train, X_test, y_train, y_test = train_test_split(positive_shadow_X, positive_shadow_y, test_size=0.2, random_state=42)

shadow_original_model = CustomLogisticRegressionModel(X_train, y_train)
shadow_original_model.train()

# Evaluate the model with testing data
test_accuracy = shadow_original_model.evaluate(X_test, y_test)

print(f"Testing accuracy: {test_accuracy}")

Testing accuracy: 0.8333333333333334


/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
#create unlearned shadow models

shadow_unlearned_models = {}

for i in range(len(positive_shadow_X)):
    temp_X = np.delete(positive_shadow_X, i, axis=0)
    temp_y = np.delete(positive_shadow_y, i, axis=0)
    
    model = LogisticRegression()
    
    model.fit(temp_X, temp_y)
    
    shadow_unlearned_models[f"shadow_unlearned_model_index_{i}"] = model

shadow_unlearned_models

/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aadya/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

{'shadow_unlearned_model_index_0': LogisticRegression(),
 'shadow_unlearned_model_index_1': LogisticRegression(),
 'shadow_unlearned_model_index_2': LogisticRegression(),
 'shadow_unlearned_model_index_3': LogisticRegression(),
 'shadow_unlearned_model_index_4': LogisticRegression(),
 'shadow_unlearned_model_index_5': LogisticRegression(),
 'shadow_unlearned_model_index_6': LogisticRegression(),
 'shadow_unlearned_model_index_7': LogisticRegression(),
 'shadow_unlearned_model_index_8': LogisticRegression(),
 'shadow_unlearned_model_index_9': LogisticRegression(),
 'shadow_unlearned_model_index_10': LogisticRegression(),
 'shadow_unlearned_model_index_11': LogisticRegression(),
 'shadow_unlearned_model_index_12': LogisticRegression(),
 'shadow_unlearned_model_index_13': LogisticRegression(),
 'shadow_unlearned_model_index_14': LogisticRegression(),
 'shadow_unlearned_model_index_15': LogisticRegression(),
 'shadow_unlearned_model_index_16': LogisticRegression(),
 'shadow_unlearned_model